In [1]:
import os

In [2]:
#I Received An Error While Using KMeans. The warning Suggested this Itself
os.environ["OMP_NUM_THREADS"] = "4"

In [3]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.cluster import KMeans
import time

In [4]:
df = pd.read_csv("titanic.csv")
df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [5]:
df.info() #I Have Some Nulls In My Dataset

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [6]:
df.describe(include="all") #Some Changes Need To Be Made In The Values

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
count,891.000000,891.000000,891.000000,891,891,714.000000,891.000000,891.000000,891,891.000000,204,889
unique,NaN,NaN,NaN,891,2,NaN,NaN,NaN,681,NaN,147,3
top,NaN,NaN,NaN,"Braund, Mr. Owen Harris",male,NaN,NaN,NaN,347082,NaN,B96 B98,S
freq,NaN,NaN,NaN,1,577,NaN,NaN,NaN,7,NaN,4,644
mean,446.000000,0.383838,2.308642,NaN,NaN,29.699118,0.523008,0.381594,NaN,32.204208,NaN,NaN
std,257.353842,0.486592,0.836071,NaN,NaN,14.526497,1.102743,0.806057,NaN,49.693429,NaN,NaN
min,1.000000,0.000000,1.000000,NaN,NaN,0.420000,0.000000,0.000000,NaN,0.000000,NaN,NaN
25%,223.500000,0.000000,2.000000,NaN,NaN,20.125000,0.000000,0.000000,NaN,7.910400,NaN,NaN
50%,446.000000,0.000000,3.000000,NaN,NaN,28.000000,0.000000,0.000000,NaN,14.454200,NaN,NaN
75%,668.500000,1.000000,3.000000,NaN,NaN,38.000000,1.000000,0.000000,NaN,31.000000,NaN,NaN


In [7]:
#Checking The Percentage Of Missing Values
print(sum(df["Age"].isna())/len(df["Age"])*100)
print(sum(df["Cabin"].isna())/len(df["Cabin"])*100)
print(sum(df["Embarked"].isna())/len(df["Embarked"])*100)

19.865319865319865
77.10437710437711
0.22446689113355783


In [8]:
#Droping The Rows That Miss Value in df["Embarked"]
df.dropna(subset=["Embarked"], inplace=True)

In [9]:
#Filling Nulls Based on Their df["Sex"] & df["Pclass"]
df["Age"] = df.groupby(["Sex", "Pclass"])["Age"].transform(lambda x: x.fillna(x.median()))

In [10]:
#Filling df["Cabin"] With Zeros
df.fillna({"Cabin" : 0}, inplace=True)

In [11]:
#Turning Catrgorical Data Into Numerics
df["Sex"] = df["Sex"].map({"male": 0, "female": 1})

In [12]:
#Preparin df["Embarked"] for train

df = pd.get_dummies(df, columns=["Embarked"])
#le = LabelEncoder()
#df["Embarked"] = le.fit_transform(df["Embarked"])

#pd.get_dummies Gave me Better Accuracy Than LabelEncoder

In [13]:
#Preparin df["Ticket"] for train

df["Ticket"] = df["Ticket"].str.extract(r"(\D*)").fillna("None")
df = pd.get_dummies(df, columns=["Ticket"])
#df["Ticket"] = le.fit_transform(df["Ticket"])

In [14]:
#Preparin df["Cabin"] for train

df["Cabin"] = df["Cabin"].str[0]
df = pd.get_dummies(df, columns=["Cabin"])
#df["Cabin"] = le.fit_transform(df["Cabin"])

In [15]:
#Finished Preprocessing
df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Embarked_C,...,Ticket_W/C,Ticket_WE/P,Cabin_A,Cabin_B,Cabin_C,Cabin_D,Cabin_E,Cabin_F,Cabin_G,Cabin_T
0,1,0,3,"Braund, Mr. Owen Harris",0,22.0,1,0,7.2500,False,...,False,False,False,False,False,False,False,False,False,False
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,38.0,1,0,71.2833,True,...,False,False,False,False,True,False,False,False,False,False
2,3,1,3,"Heikkinen, Miss. Laina",1,26.0,0,0,7.9250,False,...,False,False,False,False,False,False,False,False,False,False
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",1,35.0,1,0,53.1000,False,...,False,False,False,False,True,False,False,False,False,False
4,5,0,3,"Allen, Mr. William Henry",0,35.0,0,0,8.0500,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",0,27.0,0,0,13.0000,False,...,False,False,False,False,False,False,False,False,False,False
887,888,1,1,"Graham, Miss. Margaret Edith",1,19.0,0,0,30.0000,False,...,False,False,False,True,False,False,False,False,False,False
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",1,21.5,1,2,23.4500,False,...,False,False,False,False,False,False,False,False,False,False
889,890,1,1,"Behr, Mr. Karl Howell",0,26.0,0,0,30.0000,True,...,False,False,False,False,True,False,False,False,False,False


In [16]:
#Clusterin Values
#age = df[["Age"]]
#kmeans = KMeans(n_clusters=5, random_state=42)
#df["AgeCluster"] = kmeans.fit_predict(df[["Age"]])

#fare = df[["Fare"]]
#kmeans = KMeans(n_clusters=3, random_state=42)
#df["FareCluster"] = kmeans.fit_predict(df[["Fare"]])

#Model's Accuracy Worsened so I Just Commented them

In [17]:
#Reducing ["Parch"], ["SibSp"] to Two Values
df["Parch"] = df["Parch"].where(df["Parch"]==0, 1)
df["SibSp"] = df["SibSp"].where(df["SibSp"]==0, 1)

In [18]:
#Initializing & Training Model

X = df.drop(["Survived", "PassengerId", "Name"], axis=1)
y = df["Survived"]

X_train, X_test, y_train, y_test = train_test_split(X, y , test_size=0.2, random_state=42)
start_time = time.time()
#model = LogisticRegression(penalty="l1", solver= "liblinear", C=1.0, max_iter=1000)
model = LogisticRegression(penalty="l2", solver= "lbfgs", C=1.0, max_iter=1000)
#model = LogisticRegression(C=1.0, max_iter=1000)
model.fit(X_train, y_train)
end_time= time.time()
duration = end_time - start_time
print(duration)
#l2 Regulizer Gave Me The Best Accuracy Score

0.08644795417785645


In [19]:
#Predict
start_time = time.time()
y_pred = model.predict(X_test)
end_time= time.time()
duration = end_time - start_time
print(duration)

0.003816366195678711


In [20]:
#Evaluation
print("Accuracy:", accuracy_score(y_test, y_pred))

Accuracy: 0.8146067415730337
